# HyP3 On-Demand GAMMA-InSAR, SBAS

## Example: programmatic access of ASF HyP3 On-Demand Processing.

- Downloads interferograms from HyP3

**Author:** Jie Chen, University of Alaska Fairbanks

>**Example Only. Not Needed for Workshop.**
>- This orders the interferograms used with the accompanying notebook, `MintPy_sbas_permafrost_Alaska.ipynb` (<u>Quantifying permafrost-related ground deformation with InSAR Time Series using HyP3 products and MintPy</u>).
>- The dataset has already been processed and saved in an S3 bucket. The MintPy notebook uses the pre-staged dataset, so it is not necessary to run this notebook.

**Adapt this code to order SBAS stacks over your area of interest.**

### Set parameters

In [1]:
from pathlib import Path
from dateutil.parser import parse as parse_date

import pandas as pd
from typing import List, Union
from osgeo import gdal

import asf_search as asf
import hyp3_sdk as sdk

## 1. Select InSAR pairs with ASF Search

In [23]:
# Define a project name
project_name = 'AK_P15D_2021-2022'
work_dir = Path.cwd()
data_dir = work_dir / 'insar_data'

# Define the dataset parameters
start_date = '2021-06-01'
end_date = '2022-10-01'
max_temporal_baseline = 36  # days
interannual_baseline_max = 380
interannual_baseline_min = 350
look_nums = '20x4'     # Range x Azimuth looks
phase_filter_para = 0.8   # Exponent parameters for adaptive filter before phase unwrapping
overlap = [516000.000, 7535000.000, 526000.000, 7525000.000]   # AOI with common overlap

# Set the polygons
wkt = 'POLYGON((-164.5341 67.8529,-164.2322 67.8529,-164.2322 67.9293,-164.5341 67.9293,-164.5341 67.8529))'   # Get from ASF AOI

stack_start = parse_date(start_date + ' 00:00:00Z')
stack_end = parse_date(end_date + ' 00:00:00Z')
year_start = stack_start.year
year_end = stack_end.year
season_period = [start_date[5:], end_date[5:]]

data_dir.mkdir(parents=True, exist_ok=True)

In [24]:
# All options to geo_search() can be specified using kwargs, which also allows them to be handled using a dictionary.
opts = {
    'platform': asf.SENTINEL1,
    'start': start_date,
    'end': end_date,
    'processingLevel': asf.SLC,
    'beamMode': asf.IW,
    'flightDirection': asf.DESCENDING,
    'relativeOrbit': 15
}
search_results = asf.geo_search(
        intersectsWith=wkt,
        **opts
    )

In [25]:
centroid = search_results[1].centroid().wkt
print(centroid)
centroid_results = asf.geo_search(intersectsWith=centroid, **opts)
search_results = centroid_results
print(f'{len(search_results)} SLC images found')
# print(search_results[0])

POINT (-165.05558675728977 67.33204179507277)
41 SLC images found


In [26]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

In [27]:
# Construct interferometric pairs
stack_season = pd.DataFrame()
for year_index in range(int(year_start), int(year_end) + 1):
    season_start = parse_date(f'{str(year_index)}-{season_period[0]} 00:00:00Z')
    season_end = parse_date(f'{str(year_index)}-{season_period[1]} 00:00:00Z')
    stack_season = pd.concat([stack_season, stack.loc[(season_start <= stack.startTime) & (stack.startTime <= season_end)]])
print('Total SAR sences are:', len(stack_season.startTime))
for i in stack_season.startTime.values:   # or stack_season['startTime'].values
    print(i, type(i))

Total SAR sences are: 21
2021-06-07T17:33:39.000000000 <class 'numpy.datetime64'>
2021-06-19T17:33:40.000000000 <class 'numpy.datetime64'>
2021-07-01T17:33:41.000000000 <class 'numpy.datetime64'>
2021-07-13T17:33:41.000000000 <class 'numpy.datetime64'>
2021-07-25T17:33:42.000000000 <class 'numpy.datetime64'>
2021-08-06T17:33:43.000000000 <class 'numpy.datetime64'>
2021-08-18T17:33:43.000000000 <class 'numpy.datetime64'>
2021-08-30T17:33:44.000000000 <class 'numpy.datetime64'>
2021-09-11T17:33:45.000000000 <class 'numpy.datetime64'>
2021-09-23T17:33:45.000000000 <class 'numpy.datetime64'>
2022-06-02T17:33:45.000000000 <class 'numpy.datetime64'>
2022-06-14T17:33:46.000000000 <class 'numpy.datetime64'>
2022-06-26T17:33:47.000000000 <class 'numpy.datetime64'>
2022-07-08T17:33:47.000000000 <class 'numpy.datetime64'>
2022-07-20T17:33:48.000000000 <class 'numpy.datetime64'>
2022-08-01T17:33:49.000000000 <class 'numpy.datetime64'>
2022-08-13T17:33:50.000000000 <class 'numpy.datetime64'>
2022-0

In [28]:
# Construct the seasonal and inter-annual pairs. You can modify the networking strategies.
sbas_pairs = set()

# For adjacent interferogram pair search
sbas_pairs = set()
print('The seasonal interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= max_temporal_baseline)
        & (stack_season.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])

# For inter-annual pairs
print('The inter-annual interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    # print(reference, rt)
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= interannual_baseline_max)
        & (stack_season.temporalBaseline - rt > interannual_baseline_min)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])
print('The total number of interferometric pairs are: ', len(sbas_pairs))

The seasonal interferometric pairs are (reference-secondary): 
20220918 20220930
20220906 20220918
20220906 20220930
20220825 20220906
20220825 20220918
20220825 20220930
20220813 20220825
20220813 20220906
20220813 20220918
20220801 20220813
20220801 20220825
20220801 20220906
20220720 20220801
20220720 20220813
20220720 20220825
20220708 20220720
20220708 20220801
20220708 20220813
20220626 20220708
20220626 20220720
20220626 20220801
20220614 20220626
20220614 20220708
20220614 20220720
20220602 20220614
20220602 20220626
20220602 20220708
20210911 20210923
20210830 20210911
20210830 20210923
20210818 20210830
20210818 20210911
20210818 20210923
20210806 20210818
20210806 20210830
20210806 20210911
20210725 20210806
20210725 20210818
20210725 20210830
20210713 20210725
20210713 20210806
20210713 20210818
20210701 20210713
20210701 20210725
20210701 20210806
20210619 20210701
20210619 20210713
20210619 20210725
20210607 20210619
20210607 20210701
20210607 20210713
The inter-annual in

## 2. Request On Demand InSAR products from ASF HyP3

Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [29]:
hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username:  jiechencyz
NASA Earthdata Login password:  ········


In [30]:
print(sdk.version('hyp3_sdk'))

5.0.0


In [33]:
jobs = sdk.Batch()
print(jobs)

0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.


In [34]:
# Set up the interferogram parameters here
# This step will submit the HyP3 InSAR job! Be careful!
# The code will not check if the project is exist or not. 
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                  include_wrapped_phase= True, apply_water_mask=True,
                                  include_dem=True, include_look_vectors=True,
                                 looks=look_nums, phase_filter_parameter=phase_filter_para)

In [35]:
jobs = hyp3.watch(jobs)

  0%|          | 0/71 [timeout in 10800 s]

In [36]:
jobs = hyp3.find_jobs(name=project_name)
print(jobs)

71 HyP3 Jobs: 71 succeeded, 0 failed, 0 running, 0 pending.


In [27]:
# Check if all the jobs are succeeded.
# jobs_failed = jobs.filter_jobs(succeeded=False, running=False, failed=True, include_expired=False)
# print(jobs)

27 HyP3 Jobs: 27 succeeded, 0 failed, 0 running, 0 pending.


## 3. Subset all GeoTIFFs to AOI with common overlap

In [37]:
# Clip HyP3 geotiff products to user-defined AOI
def clip_hyp3_products_to_AOI(data_path: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to AOI with common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of AOI
    Returns: None
    """

    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_wrapped_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif', '_amp.tif']
    files_del = ["amp.tif","corr.tif","dem.tif","phi.tif", "theta.tif", "phase.tif", 'mask.tif']
    
    for extension in files_for_mintpy:
        for file in data_path.rglob(f'*{extension}'):
            dst_file = file.parent / f'{file.stem}_clip{file.suffix}'
            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)
            file.unlink()

In [38]:
# Download and clip hyp3 products one-by-one.
for index in range(0, len(jobs)):
    job = jobs[index]
    insar_product = job.download_files(data_dir)
    product_dir = sdk.util.extract_zipped_product(insar_product[0])
    clip_hyp3_products_to_AOI(product_dir, overlap)

S1AA_20210725T173342_20210830T173344_VVP036_INT80_G_weF_7D67.zip:   0%|          | 0/234975382 [00:00<?, ?it/s…

S1AA_20210830T173344_20210923T173345_VVP024_INT80_G_weF_CD67.zip:   0%|          | 0/235789495 [00:00<?, ?it/s…

S1AA_20210713T173341_20210818T173343_VVP036_INT80_G_weF_7F8E.zip:   0%|          | 0/236087284 [00:00<?, ?it/s…

S1AA_20220918T173351_20220930T173351_VVP012_INT80_G_weF_81AC.zip:   0%|          | 0/234524242 [00:00<?, ?it/s…

S1AA_20220708T173347_20220813T173350_VVP036_INT80_G_weF_083D.zip:   0%|          | 0/236161599 [00:00<?, ?it/s…

...

S1AA_20210713T173341_20210725T173342_VVP012_INT80_G_weF_157A.zip:   0%|          | 0/235933182 [00:00<?, ?it/s…

S1AA_20210607T173339_20220602T173345_VVP360_INT80_G_weF_79FF.zip:   0%|          | 0/236493911 [00:00<?, ?it/s…

S1AA_20210619T173340_20210713T173341_VVP024_INT80_G_weF_AC94.zip:   0%|          | 0/236092555 [00:00<?, ?it/s…

S1AA_20210806T173343_20220813T173350_VVP372_INT80_G_weF_7540.zip:   0%|          | 0/235540997 [00:00<?, ?it/s…

S1AA_20220720T173348_20220813T173350_VVP024_INT80_G_weF_2702.zip:   0%|          | 0/235881679 [00:00<?, ?it/s…

### Delete unused files

In [39]:
for pattern in ["xml","png","kmz","md.txt"]:
    unneeded_files = data_dir.glob(f"*/*.{pattern}")
    for file in unneeded_files:
        file.unlink()
for pattern in ["amp.tif","corr.tif","dem.tif","phi.tif", "theta.tif", "phase.tif", 'mask.tif']:
    unneeded_files = data_dir.glob(f"*/*{pattern}")
    for file in unneeded_files:
        file.unlink()